In [44]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

#from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline,make_pipeline
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import f_oneway
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler







import pandas as pd
import os
from PROJECTML import logger
import joblib # here iam saving the model because i want to save the data
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
import pickle

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import  ExtraTreesClassifier
import numpy as np



from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifierCV, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV


In [27]:


new_data=pd.read_csv("C:\\Users\\mahen\\Downloads\\Steel_industry_data.csv")


In [28]:
new_data= new_data.rename(columns={'Lagging_Current_Reactive.Power_kVarh': 'Lagging_Reactive_Power_kVarh',
                                'Leading_Current_Reactive_Power_kVarh': 'Leading_Reactive_Power_kVarh',
                                'Lagging_Current_Power_Factor': 'Lagging_Power_Factor',
                                'Leading_Current_Power_Factor': 'Leading_Power_Factor',
                                'CO2(tCO2)':'CO2'})

In [29]:
X = new_data.drop(columns=['Load_Type'])  # Features
y = new_data['Load_Type']  # Target

In [30]:
oversampler = RandomOverSampler(random_state=42)
 # Convert back to DataFrame if needed


In [31]:
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [32]:
new_data = pd.DataFrame(X_resampled, columns=X.columns)
new_data['Load_Type'] = y_resampled

        # Check the distribution after oversampling
print(new_data['Load_Type'].value_counts())
print(new_data.head())

Load_Type
Light_Load      18072
Medium_Load     18072
Maximum_Load    18072
Name: count, dtype: int64
               date  Usage_kWh  Lagging_Reactive_Power_kVarh  \
0  01/01/2018 00:15       3.17                          2.95   
1  01/01/2018 00:30       4.00                          4.46   
2  01/01/2018 00:45       3.24                          3.28   
3  01/01/2018 01:00       3.31                          3.56   
4  01/01/2018 01:15       3.82                          4.50   

   Leading_Reactive_Power_kVarh  CO2  Lagging_Power_Factor  \
0                           0.0  0.0                 73.21   
1                           0.0  0.0                 66.77   
2                           0.0  0.0                 70.28   
3                           0.0  0.0                 68.09   
4                           0.0  0.0                 64.72   

   Leading_Power_Factor   NSM WeekStatus Day_of_week   Load_Type  
0                 100.0   900    Weekday      Monday  Light_Load  
1     

In [33]:
new_data.head()
new_data['date'] = pd.to_datetime(new_data['date'], format='%d/%m/%Y %H:%M')
#new_data['date_year'] = new_data['date'].dt.year
#new_data['date_month_no'] = new_data['date'].dt.month
#new_data['date_day'] = new_data['date'].dt.day
new_data['hour'] = new_data['date'].dt.hour
#new_data['min'] = new_data['date'].dt.minute
new_data.drop(columns='date', inplace=True)  # Drop the date column since we've extracted features from it
new_data.drop(columns='Day_of_week', inplace=True)  # Drop the Day_of_week column as suggested by the ANOVA test
new_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54216 entries, 0 to 54215
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Usage_kWh                     54216 non-null  float64
 1   Lagging_Reactive_Power_kVarh  54216 non-null  float64
 2   Leading_Reactive_Power_kVarh  54216 non-null  float64
 3   CO2                           54216 non-null  float64
 4   Lagging_Power_Factor          54216 non-null  float64
 5   Leading_Power_Factor          54216 non-null  float64
 6   NSM                           54216 non-null  int64  
 7   WeekStatus                    54216 non-null  object 
 8   Load_Type                     54216 non-null  object 
 9   hour                          54216 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(2)
memory usage: 3.9+ MB


In [43]:
train, test = train_test_split(new_data,test_size=0.25,random_state=42) # this line splits the data into train_test_split
#x_train=train.drop(columns=['Load_Type'])
#y_train=train['Load_Type']
#x_test=test.drop(columns=['Load_Type'])
#y_test=test['Load_Type']



In [49]:
from sklearn.preprocessing import OrdinalEncoder

# Assuming your data is stored in a DataFrame called train and you want to encode column at index 7
column_index = 7  # Index of the column to be encoded

# Extract the column to be encoded
column_to_encode = train.iloc[:, column_index].values.reshape(-1, 1)

# Initialize OrdinalEncoder
encoder = OrdinalEncoder()

# Fit and transform the column
encoded_column = encoder.fit_transform(column_to_encode)

# Replace the original column with the encoded values
train.iloc[:, column_index] = encoded_column


In [47]:
from sklearn.preprocessing import LabelEncoder

# Assuming your data is stored in a DataFrame called train and you want to encode column at index 8
column_index = 8  # Index of the column to be encoded

# Extract the column to be encoded
column_to_encode = train.iloc[:, column_index]

# Initialize LabelEncoder
encoder = LabelEncoder()

# Fit and transform the column
encoded_column = encoder.fit_transform(column_to_encode)

# Replace the original column with the encoded values
train.iloc[:, column_index] = encoded_column


In [50]:
train.head()

,Usage_kWh,Lagging_Reactive_Power_kVarh,Leading_Reactive_Power_kVarh,CO2,Lagging_Power_Factor,Leading_Power_Factor,NSM,WeekStatus,Load_Type,hour
47189,47.77,20.09,0.00,0.02,92.18,100.00,68400,0.0,2,19
11778,122.08,49.79,0.00,0.06,92.59,100.00,60300,0.0,1,16
54172,3.31,0.00,12.20,0.00,100.00,26.18,77400,0.0,2,21
24076,3.56,0.00,17.75,0.00,100.00,19.66,69300,1.0,2,19
36552,46.91,21.46,0.00,0.02,90.94,100.00,58500,0.0,1,16


In [55]:
# Assuming your test data is stored in a DataFrame called test

# For column 7 using OrdinalEncoder
# Extract the column to be encoded
column_to_encode = test.iloc[:, 7].values.reshape(-1, 1)

# Transform the column using the already fitted encoder
encoded_column = encoder.transform(column_to_encode)

# Replace the original column with the encoded values
test.iloc[:, 7] = encoded_column




ValueError: Found unknown categories ['Weekend', 'Weekday'] in column 0 during transform

In [53]:
# For column 8 using LabelEncoder
# Extract the column to be encoded
column_to_encode = test.iloc[:, 8]

# Transform the column using the already fitted encoder
encoded_column = encoder.transform(column_to_encode)

# Replace the original column with the encoded values
test.iloc[:, 8] = encoded_column

ValueError: Expected 2D array, got 1D array instead:
array=['Medium_Load' 'Maximum_Load' 'Maximum_Load' ... 'Maximum_Load'
 'Medium_Load' 'Medium_Load'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:


x_train = train.drop(columns=['Load_Type'])
y_train = train_data['Load_Type']
x_test = test_data.drop(columns=['Load_Type'])
y_test = test_data['Load_Type']

clf = [
        LGBMClassifier(),
        RidgeClassifierCV(),
        XGBClassifier(),
        QuadraticDiscriminantAnalysis(),
        CalibratedClassifierCV(),
        BernoulliNB(),
        BaggingClassifier(),
        LogisticRegression(),
        NearestCentroid(),
        SVC(),
        LinearSVC(),
        KNeighborsClassifier(),
        GaussianNB(),
        Perceptron(),
        SGDClassifier(),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        MLPClassifier(),
        ExtraTreesClassifier(),
        AdaBoostClassifier(),
        NuSVC()
    ]

scores = []

for model in clf:
        model.fit(x_train, y_train)
        train_pred = model.predict(x_train)
        test_pred = model.predict(x_test)

        train_accuracy = accuracy_score(y_train, train_pred)
        test_accuracy = accuracy_score(y_test, test_pred)

        train_cm = confusion_matrix(y_train, train_pred)
        test_cm = confusion_matrix(y_test, test_pred)

        train_precision = precision_score(y_train, train_pred, average='weighted')
        test_precision = precision_score(y_test, test_pred, average='weighted')

        train_recall = recall_score(y_train, train_pred, average='weighted')
        test_recall = recall_score(y_test, test_pred, average='weighted')

        train_f1 = f1_score(y_train, train_pred, average='weighted')
        test_f1 = f1_score(y_test, test_pred, average='weighted')

        scores.append({
            'Model': type(model).__name__,
            'Training Accuracy': train_accuracy,
            'Testing Accuracy': test_accuracy,
            'Training Precision': train_precision,
            'Testing Precision': test_precision,
            'Training Recall': train_recall,
            'Testing Recall': test_recall,
            'Training F1-score': train_f1,
            'Testing F1-score': test_f1
        })

        print("Model:", type(model).__name__)
        print("Training Accuracy:", train_accuracy)
        print("Testing Accuracy:", test_accuracy)
        print("Training Precision:", train_precision)
        print("Testing Precision:", test_precision)
        print("Training Recall:", train_recall)
        print("Testing Recall:", test_recall)
        print("Training F1-score:", train_f1)
        print("Testing F1-score:", test_f1)

joblib.dump(model, os.path.join(config.root_dir, f"{type(model).__name__}_model.pkl"))

return pd.DataFrame(scores)


# Example usage:
class ModelTrainerConfig:
    def __init__(self, train_data_path, test_data_path, root_dir):
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.root_dir = root_dir

config = ModelTrainerConfig(
    train_data_path='train.csv',
    test_data_path='test.csv',
    root_dir='./models'
)

train(config)


In [36]:
x_train_index = x_train.columns.tolist()  # Get the column names as a list
y_train_index = y_train.name  # Get the name of the target column

print("Index values of x_train:", x_train_index)
print("Index value of y_train:", y_train_index)

Index values of x_train: ['Usage_kWh', 'Lagging_Reactive_Power_kVarh', 'Leading_Reactive_Power_kVarh', 'CO2', 'Lagging_Power_Factor', 'Leading_Power_Factor', 'NSM', 'WeekStatus', 'hour']
Index value of y_train: Load_Type


In [38]:
trf1 = ColumnTransformer(
    [('ordinal_encode', OrdinalEncoder(), [7])],  # Indices of categorical columns
    remainder='passthrough'
)

In [41]:
pipe=Pipeline([
    ('trf1',trf1)
])

In [42]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(trf1)
print(pipe)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal_encode',
                                                  OrdinalEncoder(), [7])]))])
